In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sys
import scipy
import scipy.optimize
import pyuvdata
import time
sys.path.append('/Users/ruby/Astro/rlb_LWA/dwcal')
import delay_weighted_cal as dwcal
import dwcal_tests
import importlib
importlib.reload(dwcal)
importlib.reload(dwcal_tests)
import scipy.linalg

In [2]:
x0 = np.zeros(1000, dtype=float)
a = np.random.rand(1000)
c = np.random.rand(1000,1000)
c = c+c.T

In [3]:
def cost_function(x, a, c):
    part1 = np.matmul(c, x-a)
    cost = np.matmul(np.conj(x-a).T, part1)
    return cost

def calc_jac(x, a, c):
    return 2.*np.matmul(c, x-a)

def calc_hess(x, a, c):
    return 2.*c

In [4]:
def test_jac(x, a, c, test_ind, delta=0.0001):
    x0 = np.copy(x)
    x0[test_ind] -= delta / 2.0
    x1 = np.copy(x)
    x1[test_ind] += delta / 2.0
    cost0 = cost_function(x0, a, c)
    cost1 = cost_function(x1, a, c)
    grad = calc_jac(x, a, c)
    print((cost1-cost0)/delta)
    print(grad[test_ind])
    
test_jac(x0, a, c, 0)

-977.760644746013
-977.7606445543101


In [5]:
def test_hess(x, a, c, test_ind, readout_ind, delta=0.0001):
    x0 = np.copy(x)
    x0[test_ind] -= delta / 2.0
    x1 = np.copy(x)
    x1[test_ind] += delta / 2.0
    jac0 = calc_jac(x0, a, c)
    jac1 = calc_jac(x1, a, c)
    hess = calc_hess(x, a, c)
    print((jac1[readout_ind]-jac0[readout_ind])/delta)
    print(hess[readout_ind, test_ind])
    print(hess[test_ind, readout_ind])
    
test_hess(x0, a, c, 0,1)

2.8753198921549483
2.8753198917062903
2.8753198917062903


In [6]:
def newtons_method_optimizer(
    x0,
    a,
    c,
    step_size = 1.,
    covergence_condition = 0.001,
):

    n_iters = 0
    convergence_iters = 0
    while convergence_iters < 3:
        hess_mat = calc_hess(x0, a, c)
        hess_mat_inv = np.linalg.inv(hess_mat)
        del hess_mat
        jac = calc_jac(x0, a, c)
        x1 = x0 - step_size*np.matmul(hess_mat_inv, jac)
        del hess_mat_inv
        del jac
        cost = cost_function(x0, a, c)
        print(f"Iteration {n_iters}, cost func value: {cost}")
        check_conv = np.max(np.abs(x1-x0))
        if check_conv < covergence_condition:
            convergence_iters += 1
        else:
            convergence_iters = 0
        x0 = x1
        n_iters += 1

In [7]:
newtons_method_optimizer(
    x0,
    a,
    c,
    step_size = 1.,
    covergence_condition = 0.001,
)

Iteration 0, cost func value: 234673.2056056494
Iteration 1, cost func value: -1.6446273011760608e-19
Iteration 2, cost func value: 0.0
Iteration 3, cost func value: 0.0
